# BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.ipynb), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [3]:

%reload_ext autoreload
%autoreload 2
import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["LANGCHAIN_SESSION"] = "session417"
from typing import Optional
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings


from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

from  baby_agi import BabyAGI

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

OBJECTIVE = "I have two appointments today, one at 3pm at 175 Haight St, SF and the other at 8pm at Mission Cliffs. Please find me somewhere for dinner between each where I have enough time to walk inbetween."
llm = ChatOpenAI(temperature=1, model_name="gpt-3.5-turbo")


from Chains.objective_bootstrap_chain import ObjectiveBootstrapChain
from langchain import  SerpAPIWrapper
from langchain.agents import Tool, load_tools
from google_maps.google_maps_tool import TravelTimeInfo
from google_maps.google_maps_api import GoogleMapsAPIWrapper

search = SerpAPIWrapper()
search_tool = Tool(
        name="Search",
        func=search.run,
        description="useful for when you need find businesses or places near a locations. Input: a type of business followed by the location. Output: a list of businesses or places that match the query. Example input: 'coffee shops in San Francisco'",
    )
travel_time_tool = TravelTimeInfo(api_wrapper=GoogleMapsAPIWrapper())
tools = [
    search_tool,
    travel_time_tool,
] + load_tools(['human','llm-math'], llm=llm)

In [5]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, 
    vectorstore=vectorstore, 
    verbose=verbose, 
    tools=tools,
    max_iterations=max_iterations
)

baby_agi({"objective": OBJECTIVE})
# from langchain.agents import initialize_agent
# from langchain.agents import AgentType

# agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.run(OBJECTIVE)



> Entering new ObjectiveBootstrapChain chain...
Prompt after formatting:
You are an AI travel planner who is an expert at solving travel related problems by createding steps to solve the problem and then executing them. As a planner you want to balance time, convenience, and the person's preferences. 
You have these tools available to complete the objective: Search: useful for when you need find businesses or places near a locations. Input: a type of business followed by the location. Output: a list of businesses or places that match the query. Example input: 'coffee shops in San Francisco' /n, Travel Time Info: A wrapper around Google Maps API. Useful for calculating travel time and distance between two addresses. Input should be a string seperated starting with the mode of transportation ("driving", "transit", "walking", "bicycling") followed by ':' the origin and destination addresses seperated by '<>'. For example: 'driving: New York City <> Los Angeles' If you don't know the mod

KeyboardInterrupt: 

In [ ]:
from langchain import LLMChain, PromptTemplate

from langchain.llms import BaseLLM

class ItineraryModelChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        template = """
                You are a data entry agent that must produce a valid JSON file given an input string that represents use of time for an itinerary.
                Each activity should be placed in an array of objects, where each object represents an activity.
                The object must have the following fields:

                name
                Type: String
                Description: Human reabalbe name of the activity. 

                start_time
                Type: String
                Format: "YYYY-MM-DDTHH:mm:ss"
                Description: The starting time of the activity. The time should be represented in the ISO 8601 format, which includes the date and time in the following format: "YYYY-MM-DDTHH:mm:ss".
                
                end_time
                Type: String
                Format: "YYYY-MM-DDTHH:mm:ss"
                Description: The ending time of the activity. The time should be represented in the ISO 8601 format, which includes the date and time in the following format: "YYYY-MM-DDTHH:mm:ss".
                
                location
                Type: String
                Description: The location of the activity. This field should contain a textual description of the location, such as an address or a room number, depending on the context.
                
                description
                Type: String
                Description: A brief description of the activity. This field should contain a concise summary of the activity, including its purpose, participants, or any other relevant information.

                Here is the string to generate the JSON object from: {objective}
                Only return the array of objects that represent the activities. Do not return the JSON object or include the objective.
            """
        

        human_message_prompt = PromptTemplate(
            template=template,
            input_variables=["objective"]
        )

        return cls(prompt=human_message_prompt, llm=llm, verbose=verbose)
    
chain2 = ItineraryModelChain.from_llm(llm, verbose=verbose,)
result = chain2({"objective": OBJECTIVE})
print(result)


In [ ]:
result['text']